In [1]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [2]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.processor.utils import Utils
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [4]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [5]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()

In [6]:
pm = Portfolio.from_dict(portfolio_dictionary)
print(portfolio_dictionary)

{'ranking_metric': 'option_price', 'timeframe': 'DAY', 'position_type': 'long', 'grouping_type': 'STANDARD_DEV', 'selection_type': 'top_blacklist', 'allocation_type': 'equal', 'risk_type': 'coefficient_of_variation', 'selection_percentage': 1, 'stoploss': 1, 'num_of_groups': 1, 'max_price': 1, 'min_price': 0.01, 'max_market_cap': 10000000, 'min_market_cap': 1000000, 'rolling_window': 10, 'leverage': 1, 'date': Timestamp('2024-12-30 00:00:00'), 'pnl': 1.041653163680667, 'raw_pnl': 143295.63041874088, 'downside': 0.9996193642038371, 'coefficient_of_variation': 89.50161860539842, 'portfolio_std': 0.011638372354730109, 'sharpe_ratio': -107.27934517411298, 'tracking_error': 0.32759544812677793, 'information_ratio': -3.811276903410329}


In [7]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'BRKC'
'CRCL'
'EPEM'
'EPIN'
'FFUT'
'GPZ'
'IAUI'
'KOID'
'SVA'
'TACOW'


In [8]:
end = Utils.last_weekday(alp.clock())
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-05-05 16:00:00-04:00 2025-06-04 16:00:00-04:00


50


In [13]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                price = p.lower_column(price)
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["coefficient_of_variation"] = 1
                price["dividend"] = 0
                if price.index.size > pm.rolling_window:
                    for member in Metric:
                        price = member.calculate(price,timeframe=pm.rolling_window,live=True)
                    for member in Indicator:
                        price = member.calculate(price,timeframe=pm.rolling_window,live=True)
                prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

 12%|███████████████████▍                                                                                                                                               | 27/227 [01:05<07:57,  2.39s/it]

'BRKC'


 19%|███████████████████████████████▌                                                                                                                                   | 44/227 [01:48<08:03,  2.64s/it]

'CRCL'


 28%|█████████████████████████████████████████████▏                                                                                                                     | 63/227 [02:36<07:01,  2.57s/it]

...

 87%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                     | 197/227 [08:11<01:15,  2.51s/it]

'TGE'


 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 209/227 [08:40<00:44,  2.48s/it]

'VBIX'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 227/227 [09:23<00:00,  2.48s/it]


In [14]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades.to_dict("records")

[{'year': 2025,
  'day': 155,
  'ticker': 'HBIO',
  'date': Timestamp('2025-06-04 00:00:00+0000', tz='UTC'),
  'open': 0.401,
  'adjclose': 0.4103,
  'high': 0.43,
  'low': 0.37,
  'volume': 3640659,
  'month': 6,
  'quarter': 2,
  'week': 23,
  'weekday': 2,
  'risk': 1,
  'dividend': 0.0,
  'adjclose_test': 0.4211,
  'high_test': 0.4689,
  'low_test': 0.4,
  'volume_test': 5857199.0,
  'average_return': 0.20509097328650433,
  'standard_dev': 0.1087848212451228,
  'market_cap': 1493762.3877,
  'simple_moving_average': 0.40242000000000006,
  'drawdown': 0.6334389471118694,
  'distance': 1.980023985198164,
  'cooked_return': 57.650238884309715,
  'next_close': nan,
  'next_open': nan,
  'price_to_return': 196.2153641144579,
  'optimal': nan,
  'adr': 0.2660360527092268,
  'sma': -0.019205459419936455,
  'ema': -0.0024404441314580527,
  'bollinger_upper': 1.5110642029984054,
  'bollinger_lower': 0.4505248781617218,
  'momentum': 0.07390000000000002,
  'roc': -0.02564711469959624,
  'will